<!-- ---
title: How to convert pure PyTorch code to Ignite
downloads: true
sidebar: true
tags:
  - training loop
  - validation loop
--- -->
# How to convert pure PyTorch code to Ignite 

In this guide we will demonstrate how PyTorch code compenents can be converted into compact and flexible PyTorch-Ignite code. Since Ignite focusses on the training and validation pipeline, the code for models, datasets, optimizers, etc will remain user-defined and in pure PyTorch.

## Training Loop to `trainer`

First, we need to move the code or steps taken to process a single batch of data while training under a function (`train_step()` below). This function will take `engine` and `batch` (current batch of data) as arguments and can return any data (usually the loss) that can be accessed via `engine.state.output`. We pass this function to `Engine` which creates a `trainer` object.

```python
def train_step(engine, batch):
    inputs, targets = batch
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)
```

There are other [helper methods](https://pytorch.org/ignite/engine.html#helper-methods-to-define-supervised-trainer-and-evaluator) that directly create the `trainer` object without writing a custom function for some common use cases like [supervised training](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_trainer.html#ignite.engine.create_supervised_trainer) and [truncated backprop through time](https://pytorch.org/ignite/contrib/engines.html#ignite.contrib.engines.tbptt.create_supervised_tbptt_trainer).

## Validation Loop to `evaluator`

Next we will move the model evaluation logic under another function (`validation_step()` below) which receives the same parameters as `train_step()` and processes a single batch of data to return some output (usually the predicted and actual value which can be used to calculate metrics) stored in `engine.state.output`. Another instance (called `evaluator` below) of `Engine` is created by passing the `validation_step()` function.

```python
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = prepare_batch(batch)
        y_pred = model(x)

    return y_pred, y
    
evaluator = Engine(validation_step)
```

Similar to the training loop, there are [helper methods](https://pytorch.org/ignite/engine.html#helper-methods-to-define-supervised-trainer-and-evaluator) to avoid writing this custom evaluation function like [`create_supervised_evaluator`](https://pytorch.org/ignite/generated/ignite.engine.create_supervised_evaluator.html#ignite.engine.create_supervised_evaluator).

Note that you can create different evaluators for training, validation and testing if they serve different functions. A common practice is to have two separate evaluators for training and validation, since the results of the validation evaluator is helpful in determining the best model to save.

## Switch to built-in Metrics

Then we can replace code for calculating metrics like accuracy and instead use several [out-of-the-box metrics](https://pytorch.org/ignite/metrics.html#complete-list-of-metrics) that Ignite provides or write a custom one (refer [here](https://pytorch.org/ignite/metrics.html#how-to-create-a-custom-metric)). The metrics will be computed using the `evaluator`'s output. Finally, we attach these metrics to the `evaluator`.

```python
metrics = {
  "accuracy": Accuracy()
  ...
}

for name, metric in metrics.items():
    metric.attach(evaluator, name)
```

## Organising code into Events and Handlers

Next we need to identify any code that is triggered after an event is completed. Example of events can be start of an iteration, completion or an epoch or even start of backprop. We already provide some predefined events (complete list [here](https://pytorch.org/ignite/generated/ignite.engine.events.Events.html#ignite.engine.events.Events)) however we can also create custom ones (refer [here](https://pytorch.org/ignite/concepts.html#custom-events)). We move the event specific code to different handlers (named functions, lambdas, class functions) which are attached to these events and executed whenever an event is triggered. Here are some common handlers:

### Running `evaluator` and logging metrics

When the `trainer` completes an epoch, we have to run the `evaluator` on the training/validation/test dataset to get predictions. This can be done automatically via a handler function aadded to a built-in event `EPOCH_COMPLETED` like:

```python
@trainer.on(Events.EPOCH_COMPLETED)
def run_train_validation():
    evaluator.run(train_loader)
```

### Progress Bar

Instead of a simple `tqdm` or writing a custom progress bar in the `train_step()`, we can attach a built-in `ProgressBar()` to the trainer that updates the loss returned by the `train_step()` in real time.

```python
ProgressBar().attach(trainer, output_transform=lambda x: {'batch loss': x})
```

### Checkpointing

Instead of comparing the metrics after the evaluator is run to figure out the best model, we can use the built-in `Checkpoint()` method to do it for us:

```python
def score_function(engine):
    return engine.state.metrics["accuracy"]

model_checkpoint = ModelCheckpoint(
    "checkpoints",
    n_saved=2,
    filename_prefix="best",
    score_function=score_function,
    score_name="accuracy",
    global_step_transform=global_step_from_engine(trainer),
)

evaluator.add_event_handler(Events.COMPLETED, model_checkpoint, {"model": model})
```